<a href="https://colab.research.google.com/github/rauan-assabayev/NLP/blob/master/lab2/Task2v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

In [0]:
!pip install yargy
!pip install wikipedia

In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')


START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

AS = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(eq(' :')),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    try:
        HYPONYM = eq(utils.deaccent(main_word))
        RULE = or_(
          rule(HYPONYM, ATAKJE, START, MID, END),
          rule(HYPONYM, MID, END),
          rule(START_S, END, AS, HYPONYM),
          rule(END, INCLUDING, HYPONYM),
          rule(HYPONYM, MID, START, START),
          rule(HYPONYM, MID, START),
          rule(HYPONYM, END)
        )
        parser = Parser(RULE) 
        text = utils.deaccent(wikipedia.summary(main_word))
        text = re.sub(r'\(.+?\)', '', text)
        text = text.lower()
        print(text)
        for idx, match in enumerate(parser.findall(text.lower())):
            k = [_.value for _ in match.tokens]
            print(k)
    except:
        e = sys.exc_info()[0]
        print('Not been found');

In [0]:
import pandas as pd

CHARGE_NUMBER = 7

df = pd.read_csv("https://raw.githubusercontent.com/rauan-assabayev/NLP/master/lab2/Lab2_Sheet1.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(5)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
144,NaN,145,7,Rauan Assabayev,БЛОГЕР,АВТОР,ВЛАДЕЛЕЦ,ПРОФЕССИЯ,ЖУРНАЛИСТ,ОБОЗРЕВАТЕЛЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,NaN,146,7,Rauan Assabayev,БЛОКЧЕЙН,ТЕХНОЛОГИЯ,ПРОЦЕСС,РАЗВИТИЕ,СИСТЕМА,БАЗА ДАННЫХ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,NaN,147,7,Rauan Assabayev,БОГОМОЛ,НАСЕКОМОЕ,ХИЩНИК,ЭУКАРИОТ,ОСОБЬ,ЧЛЕНИСТОНОГОЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,148,7,Rauan Assabayev,БОЙСКАУТ,СКАУТ,РАЗВЕДЧИК,ВОЕННЫЙ,МАЛЬЧИК,ЮНОША,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,NaN,149,7,Rauan Assabayev,БОЛЬШАК,ДОРОГА,ПОСЕЛОК,МЕСТО,ЛОКАЦИЯ,ГЛАВА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("-"*50)
    except: 
        pass


БЛОГЕР
гусеин гасанов  — россиискии видеоблогер .
['гусеин', 'гасанов', '—', 'россиискии', 'видеоблогер']
--------------------------------------------------
БЛОКЧЕЙН
блокчеин  — выстроенная по определенным правилам непрерывная последовательная цепочка блоков , содержащих информацию. связь между блоками обеспечивается не только нумерациеи, но и тем, что каждыи блок содержит свою собственную хеш-сумму и хеш-сумму предыдущего блока. для изменения информации в блоке придется редактировать и все последующие блоки. чаще всего копии цепочек блоков хранятся на множестве разных компьютеров независимо друг от друга. это делает краине затруднительным внесение изменении в информацию, уже включенную в блоки.
впервые термин появился как название полностью реплицированнои распределеннои базы данных, реализованнои в системе «биткоин», из-за чего блокчеин часто относят к транзакциям в различных криптовалютах, однако технология цепочек блоков может быть распространена на любые взаимосвязанные информацио